<h2>MODEL TESTING</h2>

Script ini digunakan hanya untuk mengetes model-model yang telah dibuat. Tahapan dari mengetes suatu model adalah:
1. Load model yang telah ada. (Saat ini, model yang diloat adalah model v1 yang sudah saya buat).
2. Load x_test dan y_test, untuk menjadi data yang dapat dimasukkan ke model untuk melihat performa model. Bagian ini tidak perlu diubah jika model yang diload berubah.
3. Jalankan block terakhir. Num dapat diubah-ubah untuk melihat trajectory yang berbeda-beda

In [2]:
import tensorflow as tf
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import math

In [3]:
## Load the model
model = tf.keras.models.load_model('../models/model_v4.keras')

# Show the model architecture
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ normalization_4 (Normalization) │ (None, 50, 4)          │             9 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 50, 128)        │        68,096 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 20)             │         2,580 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 10, 2)          │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 606,791 (2.31 MB)

 Trainable params: 202,260 (790.08 KB)

 Non-trainable params: 9 (40.00 B)

 Optimizer params: 404,522 (1.54 MB)

In [4]:
## Load numpy test data
x_test = np.load('../x_test_E.npy')
y_test = np.load('../y_test_E.npy')
mean_variance = np.load('../mean_variance_E.npy')

In [5]:
## IMPORTANT: Set up denormalizer
mean = mean_variance[0]
variance = mean_variance[1]

denormalizer = tf.keras.layers.Normalization(mean=mean, variance=variance, invert=True)

In [27]:
## ONE STEP PREDICTION
# Num dapat diubah-ubah untuk melihat trajectory yang berbeda-beda
num = 77

# Predict the next coords of entry "num" using the model
num_of_timesteps = len(x_test[0])
pred_coords = denormalizer(model.predict(x_test[num].reshape(1, num_of_timesteps, 4)))
true_coords = denormalizer(y_test[num])

# Print the true and predicted coordinates
print("Prediction:")
print(pred_coords)

print("True Coordinate:")
print(true_coords)


## Map display of True Point vs Predicted Point
# Get route coordinates
past_lat = []
past_lon = []

for point in x_test[num]:
  past_lat.append(point[0])
  past_lon.append(point[1])

# Get prediction coordinates
pred_lat = []
pred_lon = []

for point in pred_coords[0]:
  pred_lat.append(point[0])
  pred_lon.append(point[1])

# Get true coordinates from y
true_lat = []
true_lon = []

for point in true_coords[0]:
  true_lat.append(point[0])
  true_lon.append(point[1])


# Build the map
fig = go.Figure()

fig.add_trace(go.Scattermapbox(
  mode="markers+lines",
  lat=past_lat,
  lon=past_lon,
  name="trip",
  marker={'size': 10}))

fig.add_trace(go.Scattermapbox(
  mode="markers+lines",
  lat=pred_lat,
  lon=pred_lon,
  name="Predicted Point",
  marker={'size': 10, 'color': 'green'}))

fig.add_trace(go.Scattermapbox(
  mode="markers",
  lat=true_lat,
  lon=true_lon,
  name="True Point",
  marker={'size': 10, 'color': 'yellow'}))

fig.update_layout(mapbox_style="open-street-map",)
fig.update_layout(
  margin={"r":0,"t":0,"l":0,"b":0},
  mapbox=dict(
    zoom=10,
    center=go.layout.mapbox.Center(
            lat=-6.2,
            lon=106.816
        ),
  ))
fig.show()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step
Prediction:
tf.Tensor(
[[[ -6.2833347 106.79289  ]
  [ -6.2781177 106.78692  ]
  [ -6.272423  106.781235 ]
  [ -6.2668734 106.773506 ]
  [ -6.262746  106.776024 ]
  [ -6.2602706 106.76285  ]
  [ -6.2565527 106.764656 ]
  [ -6.2566876 106.757805 ]
  [ -6.255636  106.75954  ]
  [ -6.252349  106.75406  ]]], shape=(1, 10, 2), dtype=float32)
True Coordinate:
tf.Tensor(
[[[ -6.2920036 106.78682  ]
  [ -6.2877307 106.77762  ]
  [ -6.2791405 106.77023  ]
  [ -6.270372  106.76794  ]
  [ -6.263377  106.76727  ]
  [ -6.258964  106.76551  ]
  [ -6.256505  106.76207  ]
  [ -6.2577834 106.75967  ]
  [ -6.260703  106.75832  ]
  [ -6.261966  106.75485  ]]], shape=(1, 10, 2), dtype=float32)


In [18]:
## MULTI STEP PREDICTION
# Num dapat diubah-ubah untuk melihat trajectory yang berbeda-beda
num = 9
steps = 12

# Predict the next "step" coords of entry "num" using the model
num_of_timesteps = len(x_test[0])
x_data = x_test[num] # Data that will be fed to the model

pred_coords = []
for i in range(steps):
  current_pred = denormalizer(model.predict(x_data.reshape(1, num_of_timesteps, 4))).numpy().tolist()[0]

  pred_coords.append(current_pred)

  x_data = x_data[10:, :]

  hour_offset = math.floor(i / 60)

  hour_and_day = np.tile(np.array([x_data[-1][2] + hour_offset, x_data[-1][3]]), (10, 1))

  current_pred = np.append(current_pred, hour_and_day, axis=1)
  
  x_data = np.append(x_data, current_pred, axis=0)

true_coords = denormalizer(y_test[num])

# Print the true and predicted coordinates
print("Prediction:")
print(pred_coords)

print("True Coordinate:")
print(true_coords)


### Map display of True Point vs Predicted Point

# Get route coordinates
past_lat = []
past_lon = []
for point in x_test[num]:
  past_lat.append(point[0])
  past_lon.append(point[1])

# Get prediction coordinates
pred_lat = []
pred_lon = []
for step in pred_coords:
  for point in step:
    pred_lat.append(point[0])
    pred_lon.append(point[1])

# Get true coordinates from y
true_lat = []
true_lon = []

for point in true_coords[0]:
  true_lat.append(point[0])
  true_lon.append(point[1])

# Build the map
fig = go.Figure()

# Plot previous path
fig.add_trace(go.Scattermapbox(
  mode="markers+lines",
  lat=past_lat,
  lon=past_lon,
  name="trip",
  marker={'size': 10}))

# Plot predicted path
fig.add_trace(go.Scattermapbox(
  mode="markers+lines",
  lat=pred_lat,
  lon=pred_lon,
  name="Predicted Point",
  marker={'size': 10, 'color': 'green'}))

fig.add_trace(go.Scattermapbox(
  mode="markers",
  lat=true_lat,
  lon=true_lon,
  name="True Point",
  marker={'size': 10, 'color': 'yellow'}))

fig.update_layout(mapbox_style="open-street-map",)
fig.update_layout(
  margin={"r":0,"t":0,"l":0,"b":0},
  mapbox=dict(
    zoom=10,
    center=go.layout.mapbox.Center(
            lat=-6.2,
            lon=106.816
        ),
  ))
fig.show()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Prediction:
[[[-6.396649360656738, 106.8921890258789], [-6.41413688659668, 106.89026641845703], [-6.424880027770996, 106.88859558105469], [-6.438603401184082, 106.88932800292969], [-6.448788166046143, 106.88468933105469], [-6.462026119232178, 106.88800811767578], [-6.473902702331543, 106.88545227050781], [-6.481040954589844, 106.88176727294922], [-6.48689603805542, 106.8775405883789], [-6.492232799530029, 106.87335968017578]], [[-6.500650405883789, 106.86968231201172], [-6.501364231109619, 106.87249755859375], [-6.503398895263672, 106.864